# Agentic RAG


In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
import nest_asyncio
nest_asyncio.apply()
from loguru import logger

In [2]:
import os

QDRANT_API_KEY = os.environ.get("QDRANT_API_KEY", "")
QDRANT_HOSTED_URL = os.environ.get("QDRANT_HOSTED_URL", "")
QDRANT_PORT = os.environ.get("QDRANT_PORT", 6333)

len(QDRANT_API_KEY), QDRANT_API_KEY[:10], len(QDRANT_HOSTED_URL), QDRANT_HOSTED_URL[:10], QDRANT_PORT, type(QDRANT_PORT)

(54, 'PM4CDFLF-s', 75, 'https://94', 6333, int)

In [3]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

llm = OpenAI(
    model="gpt-4o",
)
embedding_model = OpenAIEmbedding(
    model="text-embedding-3-large",
)

In [5]:
from llama_index.core import VectorStoreIndex
from llama_index.core.chat_engine.types import ChatMode
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from llama_index.vector_stores.qdrant import QdrantVectorStore
import qdrant_client

COLLECTION_NAME = "trailhead"
q_client= qdrant_client.QdrantClient(
    url=QDRANT_HOSTED_URL, 
    port=QDRANT_PORT, 
    api_key=QDRANT_API_KEY
)

trailhead_vector_store = QdrantVectorStore(
    collection_name=COLLECTION_NAME,
    client=q_client,
    prefer_grpc=True,
    parallel=8,
    enable_hybrid=False
)

trailhead_index = VectorStoreIndex.from_vector_store(
    vector_store=trailhead_vector_store,
    embed_model=embedding_model,
    show_progress=True
)

In [6]:
trailhead_vector_store._collection_exists(COLLECTION_NAME)

True

## Create Query Engine


In [8]:
from llama_index.core.response_synthesizers import get_response_synthesizer, ResponseMode
trailhead_engine = trailhead_index.as_chat_engine(
    chat_mode=ChatMode.BEST,
    llm=llm,
    similarity_top_k=8,
    vector_store_query_mode=VectorStoreQueryMode.MMR,
    vector_store_kwargs={
        "mmr_prefetch_k": 16,
    },
    response_synthesizer=get_response_synthesizer(
        response_mode=ResponseMode.TREE_SUMMARIZE,
    )
)

In [9]:
response = trailhead_engine.chat("Which statement best describes a primary key?")
type(response), vars(response)

(llama_index.core.chat_engine.types.AgentChatResponse,
 {'response': 'A primary key is a unique identifier for each record in a database table, ensuring that each row in the table is uniquely identified and facilitating the establishment of relationships between different tables in a database.',
  'sources': [ToolOutput(content='A primary key is a unique identifier for each record in a database table, ensuring that each row in the table is uniquely identified and facilitating the establishment of relationships between different tables in a database.', tool_name='query_engine_tool', raw_input={'input': 'Which statement best describes a primary key?'}, raw_output=Response(response='A primary key is a unique identifier for each record in a database table, ensuring that each row in the table is uniquely identified and facilitating the establishment of relationships between different tables in a database.', source_nodes=[NodeWithScore(node=TextNode(id_='05575ffb-1d33-4975-a418-bcf14f953ca7'

In [10]:
from pprint import pprint
pprint(response.response)

('A primary key is a unique identifier for each record in a database table, '
 'ensuring that each row in the table is uniquely identified and facilitating '
 'the establishment of relationships between different tables in a database.')


In [16]:
print("\n".join([str(n.score) for n in response.source_nodes]))

0.19617781
0.1892829
0.1852832
0.183128
0.18294527
0.18075332
0.17942542
0.17303854


In [18]:
pprint(response.source_nodes[0].metadata)

{'creation_date': '2025-03-22',
 'document_title': '"Comprehensive Technical Documentation and Security '
                   'Analysis for B2C Projects: Architecture, Data Models, and '
                   'CumulusCI Setup"',
 'excerpt_keywords': 'file path, encryption, MACOSX directory, SQL insights, '
                     'security analysis',
 'file_name': 'tmpmpxmzw01.zip',
 'file_path': '/var/folders/9v/753h998178382xkzg96v6mwr0000gn/T/tmpmpxmzw01.zip',
 'file_size': 4396855,
 'file_type': 'application/zip',
 'last_modified_date': '2025-03-22',
 'questions_this_excerpt_can_answer': '1. What is the file path of the '
                                      'temporary ZIP file mentioned in the '
                                      'text chunk context?\n'
                                      '2. Can you identify any specific '
                                      'patterns or sequences within the '
                                      'encoded text that might suggest a '
           